In [1]:
import os
import random
import numpy as np
import tensorflow as tf

from tensorflow.keras import layers, models, losses, optimizers, metrics

from model_train import load_i3d_model, supervised_contrastive_loss
from data_uniform_sup import VideoDataGenerator
from paired_generator import PairedDataGenerator

In [2]:
seed = 2042
np.random.seed(seed)
random.seed(seed)
tf.random.set_seed(seed)

In [3]:
# Define dataset paths
violence_negative_dir = './data/violence_dataset/NonViolence'
violence_positive_dir = './data/violence_dataset/Violence'
tiktok_negative_dir = './data/tiktok/train/Safe'
tiktok_positive_dir = './data/tiktok/train/Harmful Content'


In [4]:
def sample_videos(directory, num_samples=100):
    all_videos = [os.path.join(directory, f) for f in os.listdir(directory) if f.endswith('.mp4')]
    sampled_videos = random.sample(all_videos, min(num_samples, len(all_videos)))
    return sampled_videos


In [5]:
# Violence dataset
violence_negative_videos = sample_videos(violence_negative_dir, 100)
violence_positive_videos = sample_videos(violence_positive_dir, 100)

# TikTok dataset
tiktok_negative_videos = sample_videos(tiktok_negative_dir, 100)
tiktok_positive_videos = sample_videos(tiktok_positive_dir, 100)


In [6]:
def split_data(negative_videos, positive_videos, train_ratio=0.55, val_ratio=0.15):
    # Combine and shuffle
    videos = negative_videos + positive_videos
    labels = [0]*len(negative_videos) + [1]*len(positive_videos)
    combined = list(zip(videos, labels))
    random.shuffle(combined)
    videos[:], labels[:] = zip(*combined)
    
    # Calculate split indices
    total = len(videos)
    train_end = int(total * train_ratio)
    val_end = train_end + int(total * val_ratio)
    
    # Split data
    train_videos = videos[:train_end]
    train_labels = labels[:train_end]
    val_videos = videos[train_end:val_end]
    val_labels = labels[train_end:val_end]
    test_videos = videos[val_end:]
    test_labels = labels[val_end:]
    
    return (train_videos, train_labels), (val_videos, val_labels), (test_videos, test_labels)


In [7]:
# Violence dataset
(violence_train_videos, violence_train_labels), \
(violence_val_videos, violence_val_labels), \
(violence_test_videos, violence_test_labels) = split_data(violence_negative_videos, violence_positive_videos)

# TikTok dataset
(tiktok_train_videos, tiktok_train_labels), \
(tiktok_val_videos, tiktok_val_labels), \
(tiktok_test_videos, tiktok_test_labels) = split_data(tiktok_negative_videos, tiktok_positive_videos)


In [8]:
from tensorflow.keras.utils import to_categorical

# Convert labels to numpy arrays and one-hot encode them if necessary
def prepare_labels(labels):
    return np.array(labels)

# Violence dataset generators
violence_train_labels_np = prepare_labels(violence_train_labels)
violence_val_labels_np = prepare_labels(violence_val_labels)
violence_test_labels_np = prepare_labels(violence_test_labels)

violence_train_generator = VideoDataGenerator(violence_train_videos, violence_train_labels_np, batch_size=4, shuffle=True, augment=True)
violence_val_generator = VideoDataGenerator(violence_val_videos, violence_val_labels_np, batch_size=4, shuffle=False)
violence_test_generator = VideoDataGenerator(violence_test_videos, violence_test_labels_np, batch_size=4, shuffle=False)

In [9]:

# TikTok dataset generators
tiktok_train_labels_np = prepare_labels(tiktok_train_labels)
tiktok_val_labels_np = prepare_labels(tiktok_val_labels)
tiktok_test_labels_np = prepare_labels(tiktok_test_labels)

tiktok_train_generator = VideoDataGenerator(tiktok_train_videos, tiktok_train_labels_np, batch_size=4, shuffle=True, augment=True)
tiktok_val_generator = VideoDataGenerator(tiktok_val_videos, tiktok_val_labels_np, batch_size=4, shuffle=False)
tiktok_test_generator = VideoDataGenerator(tiktok_test_videos, tiktok_test_labels_np, batch_size=4, shuffle=False)


In [10]:

input_shape = (64, 224, 224, 3)  # As defined in your data generator
num_classes = 2  # Harmful or Safe

# Load the model
model = load_i3d_model(input_shape=input_shape)


In [11]:
# Modify the model to output class probabilities


# Freeze the base model if desired
for layer in model.layers:
    layer.trainable = False

# Add classification layer
features = model.output
outputs = layers.Dense(num_classes, activation='softmax')(features)
classification_model = models.Model(inputs=model.input, outputs=outputs)

# Compile the model
classification_model.compile(
    loss=losses.SparseCategoricalCrossentropy(),
    optimizer=optimizers.Adam(learning_rate=1e-4),
    metrics=[metrics.SparseCategoricalAccuracy()]
)


In [12]:
# Train on Violence dataset
history_violence = classification_model.fit(
    violence_train_generator,
    validation_data=violence_val_generator,
    epochs=10
)


Epoch 1/10
27/27 [==============================] - 36s 898ms/step - loss: 1.4910 - sparse_categorical_accuracy: 0.3981 - val_loss: 1.1193 - val_sparse_categorical_accuracy: 0.5000
Epoch 2/10
27/27 [==============================] - 22s 824ms/step - loss: 1.3366 - sparse_categorical_accuracy: 0.4259 - val_loss: 0.9518 - val_sparse_categorical_accuracy: 0.5357
Epoch 3/10
27/27 [==============================] - 22s 824ms/step - loss: 1.3417 - sparse_categorical_accuracy: 0.4444 - val_loss: 0.8691 - val_sparse_categorical_accuracy: 0.5714
Epoch 4/10
27/27 [==============================] - 22s 830ms/step - loss: 1.3384 - sparse_categorical_accuracy: 0.4167 - val_loss: 0.7444 - val_sparse_categorical_accuracy: 0.5714
Epoch 5/10
27/27 [==============================] - 22s 816ms/step - loss: 1.1201 - sparse_categorical_accuracy: 0.4352 - val_loss: 0.6480 - val_sparse_categorical_accuracy: 0.7500
Epoch 6/10
27/27 [==============================] - 22s 812ms/step - loss: 1.0285 - sparse_cate

In [13]:
# Evaluate on Violence test set
results_violence = classification_model.evaluate(violence_test_generator)
print(f"Violence Dataset - Test Loss: {results_violence[0]}, Test Accuracy: {results_violence[1]}")


15/15 [==============================] - 8s 484ms/step - loss: 0.8110 - sparse_categorical_accuracy: 0.5000
Violence Dataset - Test Loss: 0.8109877705574036, Test Accuracy: 0.5


In [12]:
history_tiktok = classification_model.fit(
    tiktok_train_generator,
    validation_data=tiktok_val_generator,
    epochs=10
)

# Evaluate on TikTok test set
results_tiktok = classification_model.evaluate(tiktok_test_generator)
print(f"TikTok Dataset - Test Loss: {results_tiktok[0]}, Test Accuracy: {results_tiktok[1]}")

Epoch 1/10
27/27 [==============================] - 259s 9s/step - loss: 0.9250 - sparse_categorical_accuracy: 0.5324 - val_loss: 0.9092 - val_sparse_categorical_accuracy: 0.5357
Epoch 2/10
 8/27 [=======>......................] - ETA: 2:31 - loss: 0.8371 - sparse_categorical_accuracy: 0.5000


KeyboardInterrupt



## MSupCL implementation

In [ ]:
# Combine training data from both datasets
combined_train_videos = violence_train_videos + tiktok_train_videos
combined_train_labels = violence_train_labels_np.tolist() + tiktok_train_labels_np.tolist()

# Create a combined data generator
combined_train_generator = VideoDataGenerator(combined_train_videos, combined_train_labels, batch_size=4, shuffle=True, augment=True)


In [ ]:


# Create separate generators for violence and tiktok datasets
violence_train_generator_no_aug = VideoDataGenerator(violence_train_videos, violence_train_labels_np, batch_size=4, shuffle=True, augment=False)
tiktok_train_generator_no_aug = VideoDataGenerator(tiktok_train_videos, tiktok_train_labels_np, batch_size=4, shuffle=True, augment=False)

# Create paired data generator
paired_train_generator = PairedDataGenerator(violence_train_generator_no_aug, tiktok_train_generator_no_aug)




In [ ]:
def train_msupcl_model(model, paired_train_generator, epochs=10, temperature=0.07):
    optimizer = optimizers.Adam(learning_rate=1e-4)
    for epoch in range(epochs):
        print(f"Epoch {epoch + 1}/{epochs}")
        epoch_loss_avg = tf.keras.metrics.Mean()
        for step in range(len(paired_train_generator)):
            (X1_batch, X2_batch), y_batch = paired_train_generator[step]
            with tf.GradientTape() as tape:
                features1 = model(X1_batch, training=True)
                features2 = model(X2_batch, training=True)
                # Concatenate features and labels
                features = tf.concat([features1, features2], axis=0)
                labels = tf.concat([y_batch, y_batch], axis=0)
                loss = supervised_contrastive_loss(labels, features, temperature)
            gradients = tape.gradient(loss, model.trainable_variables)
            optimizer.apply_gradients(zip(gradients, model.trainable_variables))
            epoch_loss_avg.update_state(loss)
        print(f"Training Loss: {epoch_loss_avg.result():.4f}")


In [ ]:
# Load the model
msupcl_model = load_i3d_model(input_shape=input_shape)

# Train the model
train_msupcl_model(msupcl_model, paired_train_generator, epochs=10)


In [ ]:
def linear_evaluation(model, train_generator, val_generator, num_classes=2):
    # Freeze the base model
    for layer in model.layers:
        layer.trainable = False
    
    # Add a classification head
    features = model.output
    outputs = layers.Dense(num_classes, activation='softmax')(features)
    classifier_model = models.Model(inputs=model.input, outputs=outputs)
    
    # Compile the classifier
    classifier_model.compile(
        loss=losses.SparseCategoricalCrossentropy(),
        optimizer=optimizers.Adam(learning_rate=1e-4),
        metrics=[metrics.SparseCategoricalAccuracy()]
    )
    
    # Train the classifier on the combined dataset
    classifier_model.fit(
        train_generator,
        validation_data=val_generator,  # or any validation generator
        epochs=5
    )
    
    # Evaluate on test sets
    print("Evaluating on Violence Test Set:")
    results_violence = classifier_model.evaluate(violence_test_generator)
    print(f"Violence Test Loss: {results_violence[0]}, Test Accuracy: {results_violence[1]}")
    
    print("Evaluating on TikTok Test Set:")
    results_tiktok = classifier_model.evaluate(tiktok_test_generator)
    print(f"TikTok Test Loss: {results_tiktok[0]}, Test Accuracy: {results_tiktok[1]}")


In [ ]:
linear_evaluation(msupcl_model, combined_train_generator, violence_val_generator)
